In [1]:
import pandas as pd
from prophet import Prophet
import plotly.express as px
import plotly.graph_objects as go
import kaleido

In [2]:
df = pd.read_csv("data/processed/global_panel.csv")

In [3]:
country = df['country'].unique()[0]

In [4]:
df_c = df[df['country'] == country][['year','co2']].dropna()
df_c = df_c.rename(columns={'year':'ds','co2':'y'})
df_c['ds'] = pd.to_datetime(df_c['ds'], format='%Y')

In [5]:
def forecast_country(df, country, years=15):
    df_c = df[df['country'] == country][['year','co2']].dropna()
    df_c = df_c.rename(columns={'year':'ds','co2':'y'})
    df_c['ds'] = pd.to_datetime(df_c['ds'], format='%Y')

    m = Prophet(yearly_seasonality=False)
    m.fit(df_c)

    future = m.make_future_dataframe(periods=years, freq='YE')
    forecast = m.predict(future)
    return df_c, forecast


In [6]:
m = Prophet(yearly_seasonality=False, daily_seasonality=False, weekly_seasonality=False)
m.fit(df_c)

18:33:00 - cmdstanpy - INFO - Chain [1] start processing
18:33:00 - cmdstanpy - INFO - Chain [1] done processing


In [7]:
future = m.make_future_dataframe(periods=15, freq='YE')
forecast = m.predict(future)

In [8]:
country = "China"

df_c, forecast = forecast_country(df, country, years=15)

fig = go.Figure()

# actual
fig.add_trace(go.Scatter(x=df_c['ds'], y=df_c['y'],
                         mode='markers+lines', name='Actual'))

# forecast
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'],
                         mode='lines', name='Forecast',  line=dict(color='#E66C37')))

# uncertainty
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'],
                         mode='lines', name='Upper bound',
                         line=dict(dash='dash', color='lightgrey')))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'],
                         mode='lines', name='Lower bound',
                         line=dict(dash='dash', color='lightgrey'),
                         fill='tonexty', fillcolor='rgba(200,200,200,0.2)'))

fig.update_layout(title=f"CO₂ Emissions Forecast for {country}",
                  xaxis_title="Year", yaxis_title="CO₂",
                    paper_bgcolor='#171717',
                    plot_bgcolor='#171717',
                    font=dict(family='Segoe UI', color='white'),
                    xaxis=dict(tickfont=dict(color='white'), title_font=dict(color='white')),
                    yaxis=dict(tickfont=dict(color='white'), title_font=dict(color='white')),
                    legend=dict(
                        orientation='h',
                        x=0.5,
                        y=-0.2,
                        xanchor='center',
                        yanchor='top',
                        bgcolor= 'rgba(26,26,26,0.001)',
                        bordercolor='rgba(26,26,26,0.001)',
                        borderwidth=1,
                        font=dict(color='white')))

fig.write_image("co2_forecast_China.png", width=800, height=552)
fig.show()


18:33:03 - cmdstanpy - INFO - Chain [1] start processing
18:33:03 - cmdstanpy - INFO - Chain [1] done processing
